# Combine national and NERC data
Take the various file (EIA facilities, EIA state-level totals, EPA emissions, etc) and combine them to determine total generation, CO₂ emissions, etc nationally and at NERC regions.

In [1]:
import pandas as pd
import numpy as np
import os
from os.path import join
import sys
import json

idx = pd.IndexSlice
cwd = os.getcwd()
data_path = join(cwd, '..', 'Data storage')

In [2]:
%load_ext watermark
%watermark -iv -v

pandas      0.22.0
numpy       1.14.1
json        2.0.9
CPython 3.6.3
IPython 6.2.1


In [3]:
# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

In [4]:
# add the 'src' directory as one where we can import modules
src_dir = join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

In [5]:
%aimport Data.make_data
from Data.make_data import states_in_nerc
%aimport Analysis.index
from Analysis.index import facility_emission_gen, group_facility_data
%aimport Analysis.index
from Analysis.index import facility_co2, adjust_epa_emissions, group_fuel_cats
%aimport util.utils
from util.utils import rename_cols, add_facility_location

## Load data

In [6]:
cwd = os.getcwd()
path = join(data_path, 'Facility gen fuels and CO2 2018-03-06.csv')
eia_fac = pd.read_csv(path)

In [7]:
rename_cols(eia_fac)

In [8]:
eia_fac.head()

,f,fuel,month,plant id,total fuel (mmbtu),year,generation (mwh),elec fuel (mmbtu),geography,last_updated,lat,lon,prime mover,datetime,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg)
0,M,DFO,12,1001,1116.0,2017,114.587,1116.0,USA-IN,2018-02-28T02:03:13-05:00,39.9242,-87.4244,ALL,2017-12-01,4,81646.56,81646.56,81646.56,81646.56
1,M,DFO,11,1001,2772.0,2017,265.428,2772.0,USA-IN,2018-02-28T02:03:13-05:00,39.9242,-87.4244,ALL,2017-11-01,4,202799.52,202799.52,202799.52,202799.52
2,M,DFO,10,1001,4819.0,2017,460.695,4819.0,USA-IN,2018-02-28T02:03:13-05:00,39.9242,-87.4244,ALL,2017-10-01,4,352558.04,352558.04,352558.04,352558.04
3,M,DFO,9,1001,2720.0,2017,300.705,2720.0,USA-IN,2018-02-28T02:03:13-05:00,39.9242,-87.4244,ALL,2017-09-01,3,198995.20,198995.20,198995.20,198995.20
4,M,DFO,8,1001,6273.0,2017,609.358,6273.0,USA-IN,2018-02-28T02:03:13-05:00,39.9242,-87.4244,ALL,2017-08-01,3,458932.68,458932.68,458932.68,458932.68


In [10]:
path = join(data_path, 'Derived data',
            'Monthly EPA emissions 2018-03-06.csv')
epa = pd.read_csv(path)

## Adjusted facility emissions and generation
I have both the state and custom fuel categories here, but am only using the state fuel categories for now.

In [11]:
fuel_cat_folder = join(data_path, 'Fuel categories')
state_cats_path = join(fuel_cat_folder, 'State_facility.json')

with open(state_cats_path, 'r') as f:
    state_fuel_cat = json.load(f)
    
custom_cats_path = join(fuel_cat_folder, 'Custom_results.json')
with open(custom_cats_path, 'r') as f:
    custom_fuel_cat = json.load(f)

In [14]:
co2, gen_fuels_custom = facility_emission_gen(eia_facility=eia_fac, epa=epa,
                                              state_fuel_cat=state_fuel_cat,
                                              custom_fuel_cat=custom_fuel_cat,
                                              export_state_cats=False)

Renaming columns
Grouping facilities
Adjusting EPA emissions
Caculating CO2
Gen/fuels to state categories
Gen/fuels to custom categories


In [15]:
co2, gen_fuels_state = facility_emission_gen(eia_facility=eia_fac, epa=epa,
                                              state_fuel_cat=state_fuel_cat,
                                              custom_fuel_cat=custom_fuel_cat,
                                              export_state_cats=True)

Renaming columns
Grouping facilities
Adjusting EPA emissions
Caculating CO2
Gen/fuels to state categories


In [16]:
co2.head()

,year,month,plant id,final co2 (kg)
0,2001,1,2,0.000000e+00
1,2001,1,3,9.625411e+08
2,2001,1,4,0.000000e+00
3,2001,1,7,6.700092e+07
4,2001,1,8,7.530204e+08


In [17]:
gen_fuels_state.head()

,type,year,month,plant id,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),lat,lon,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg)
0,COW,2001,1,3,8275496.0,852306.0,8275496.0,31.006900,-88.010300,1,772103776.8,772103776.8,772103776.8,772103776.8
1,COW,2001,1,7,587162.0,45991.0,587162.0,34.012800,-85.970800,1,54782214.6,54782214.6,54782214.6,54782214.6
2,COW,2001,1,8,6910786.0,706857.0,6910786.0,33.644344,-87.196486,1,644776333.8,644776333.8,644776333.8,644776333.8
3,COW,2001,1,10,3108892.0,320454.0,3108892.0,32.601700,-87.781100,1,290059623.6,290059623.6,290059623.6,290059623.6
4,COW,2001,1,26,9745127.0,1005460.0,9745127.0,33.244211,-86.458056,1,909220349.1,909220349.1,909220349.1,909220349.1


### Check generation and fuel consumption totals

Interesting - there is some small part of generation that I'm losing along the way. It's 5 orders of magnitude smaller than the total though.

In [18]:
eia_fac['generation (mwh)'].sum()

67601240020.031395

In [19]:
gen_fuels_state['generation (mwh)'].sum()

67601426136.03137

Fuel consumption is pretty identical though

In [20]:
eia_fac['total fuel (mmbtu)'].sum(), eia_fac['elec fuel (mmbtu)'].sum()

(707222059073.79, 664121418912.0795)

In [21]:
gen_fuels_state['total fuel (mmbtu)'].sum(), gen_fuels_state['elec fuel (mmbtu)'].sum()

(707222059073.79, 664121418912.0798)

## Extra gen/fuels from non-reporting

In [22]:
%aimport Analysis.index
from Analysis.index import extra_emissions_gen

Total EIA generation/fuel consumption and emission factors

In [23]:
cwd = os.getcwd()
path = join(data_path,
            'EIA country-wide gen fuel CO2 2017-08-31.csv')
eia_total = pd.read_csv(path)

path = join(data_path,
            'Final emission factors.csv')
ef = pd.read_csv(path, index_col=0)

In [24]:
eia_total.head()

,end,f,geography,last_updated,sector,series_id,start,type,units,year,month,generation (MWh),datetime,quarter,total fuel (mmbtu),elec fuel (mmbtu),all fuel CO2 (kg),elec fuel CO2 (kg)
0,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,6,2524250.11,2017-06-01,2,NaN,NaN,0.0,0.0
1,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,5,2479480.14,2017-05-01,2,NaN,NaN,0.0,0.0
2,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,4,2267811.28,2017-04-01,2,NaN,NaN,0.0,0.0
3,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,3,2054274.30,2017-03-01,1,NaN,NaN,0.0,0.0
4,201706,M,USA,2017-08-24T11:46:12-04:00,99,ELEC.GEN.DPV-US-99.M,201401,DPV,thousand megawatthours,2017,2,1449156.85,2017-02-01,1,NaN,NaN,0.0,0.0


### Calculate CO₂, generation, and fuel consumption that is not captured by facilities

In [25]:
extra_co2, extra_gen_fuel = extra_emissions_gen(gen_fuels_state, eia_total, ef)

Results match what I have previously found in the notebooks up on GitHub (Emissions Index repo)

In [26]:
extra_gen_fuel.head()

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
COW  2001 1              16918390.0       1011665.061         13157576.0
          2              14968346.0        911346.865         11574491.0
          3              18321543.0       1153328.913         14765162.0
          4              13530117.0        776341.623         10374008.0
          5              14014024.0        801266.923         10920463.0

In [27]:
extra_gen_fuel.loc[idx['WND',:,:]]

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
WND  2001 1           -3.446894e+06      5.566881e+04      -3.446894e+06
          2           -3.288177e+06      1.130211e+05      -3.288177e+06
          3           -4.798975e+06      6.768812e+04      -4.798975e+06
          4           -6.166244e+06      8.794247e+04      -6.166244e+06
          5           -5.694490e+06      8.393153e+04      -5.694490e+06
          6           -6.120034e+06      7.735354e+04      -6.120034e+06
          7           -5.827621e+06      7.101049e+04      -5.827621e+06
          8           -5.409429e+06      5.369600e+04      -5.409429e+06
          9           -4.550627e+06      4.983153e+04      -4.550627e+06
          10          -5.668965e+06      5.820182e+04      -5.668965e+06
          11          -4.551249e+06      2.999030e+04      -4.551249e+06
          12          -6.077985e+06      2.744590e+04      -6.077985e+06
     2002 1           -8.237968e+06      1.614509e+03      -8.237968e+06
          2           -7.247621e+06      1.495079e+03      -7.247621e+06
          3           -8.650071e+06      1.805048e+03      -8.650071e+06
          4           -1.040601e+07      1.512223e+03      -1.040601e+07
          5           -1.094733e+07      1.524876e+03      -1.094733e+07
          6           -1.143913e+07      1.855552e+03      -1.143913e+07
          7           -9.036664e+06      2.156177e+03      -9.036664e+06
          8           -9.914911e+06      2.061015e+03      -9.914911e+06
          9           -7.466379e+06      1.887244e+03      -7.466379e+06
          10          -7.453861e+06      1.644759e+03      -7.453861e+06
          11          -6.657461e+06      1.521470e+03      -6.657461e+06
          12          -7.665064e+06      1.722605e+03      -7.665064e+06
     2003 1           -6.475789e+06      4.380000e-01      -6.475789e+06
          2           -7.633388e+06     -2.490000e-01      -7.633388e+06
          3           -1.061074e+07     -2.510000e-01      -1.061074e+07
          4           -1.119102e+07     -2.110000e-01      -1.119102e+07
          5           -1.030637e+07     -2.610000e-01      -1.030637e+07
          6           -1.072381e+07     -3.660000e-01      -1.072381e+07
...                             ...               ...                ...
     2015 7           -1.273987e+08      4.591517e+03      -1.273987e+08
          8           -1.218547e+08      4.083333e+03      -1.218547e+08
          9           -1.299189e+08      3.027033e+04      -1.299189e+08
          10          -1.526123e+08      3.567217e+03      -1.526123e+08
          11          -1.833724e+08      4.449315e+03      -1.833724e+08
          12          -1.872415e+08      5.930371e+03      -1.872415e+08
     2016 1           -1.704424e+08      6.929376e+04      -1.704424e+08
          2           -1.858746e+08      6.978208e+04      -1.858746e+08
          3           -2.024797e+08      4.689116e+04      -2.024797e+08
          4           -1.919616e+08     -4.826842e+04      -1.919616e+08
          5           -1.739356e+08     -4.504118e+04      -1.739356e+08
          6           -1.504557e+08      2.118398e+04      -1.504557e+08
          7           -1.625816e+08     -1.605398e+04      -1.625816e+08
          8           -1.254098e+08     -2.362938e+04      -1.254098e+08
          9           -1.514009e+08      3.079536e+04      -1.514009e+08
          10          -1.876898e+08      5.003013e+04      -1.876898e+08
          11          -1.791183e+08     -5.966200e+04      -1.791183e+08
          12          -2.136343e+08     -1.495990e+05      -2.136343e+08
     2017 1           -7.033578e+07      1.273091e+07      -7.033578e+07
          2           -7.382165e+07      1.369535e+07      -7.382165e+07
          3           -8.841420e+07      1.602198e+07      -8.841420e+07
          4           -8.743995e+07      1.593169e+07      -8.743995e+07
  

## Total CO₂ (national)
Combine adjusted CO₂ at facilities and CO₂ from fuel consumption that is not captured by facility data.

In [28]:
facility_co2 = co2.groupby(['year', 'month']).sum()

In [29]:
facility_co2.tail()

plant id  final co2 (kg)
year month                          
2017 8      59956777    1.685669e+11
     9      60081740    1.386751e+11
     10     60498933    1.275088e+11
     11     61215677    1.233808e+11
     12     62726265    1.438485e+11

In [30]:
extra_co2.loc[idx['NG', :, :],:].tail(n=11)

all fuel co2 (kg)  elec fuel co2 (kg)
type year month                                       
NG   2017 2           6.649623e+09        5.701198e+09
          3           8.718188e+09        7.673005e+09
          4           7.898607e+09        6.928326e+09
          5           8.813928e+09        7.892015e+09
          6           1.004648e+10        9.183183e+09
          7          -5.098617e+10       -4.742857e+10
          8          -4.944121e+10       -4.598322e+10
          9          -4.193022e+10       -3.855699e+10
          10         -3.832656e+10       -3.489319e+10
          11         -3.337851e+10       -2.995428e+10
          12         -3.881201e+10       -3.500417e+10

In [31]:
national_co2 = (facility_co2.loc[:, 'final co2 (kg)']
                + extra_co2.loc[:, 'elec fuel co2 (kg)'].groupby(['year', 'month']).sum())

## National Index and gen by fuels

In [32]:
extra_gen_fuel.head()

total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)
type year month                                                         
COW  2001 1              16918390.0       1011665.061         13157576.0
          2              14968346.0        911346.865         11574491.0
          3              18321543.0       1153328.913         14765162.0
          4              13530117.0        776341.623         10374008.0
          5              14014024.0        801266.923         10920463.0

In [33]:
gen_fuels_state.groupby(['year', 'month']).sum().tail()

plant id  total fuel (mmbtu)  generation (mwh)  elec fuel (mmbtu)  \
year month                                                                      
2017 8      92159919        3.243524e+09      3.303634e+08       3.099425e+09   
     9      92395070        2.821822e+09      2.849385e+08       2.685294e+09   
     10     92852773        2.675856e+09      2.683948e+08       2.533375e+09   
     11     93574176        2.602604e+09      2.601150e+08       2.458400e+09   
     12     95089638        2.969232e+09      2.981346e+08       2.813103e+09   

                      lat            lon  quarter  all fuel fossil co2 (kg)  \
year month                                                                    
2017 8      123777.238892 -303729.803934     9816              1.731163e+11   
     9      123966.011103 -304186.929849     9825              1.436289e+11   
     10     124485.258253 -305617.871689    13160              1.329022e+11   
     11     124758.874949 -306277.678952    13188              1.287741e+11   
     12     125632.300180 -308773.344081    13288              1.498461e+11   

            elec fuel fossil co2 (kg)  all fuel total co2 (kg)  \
year month                                                       
2017 8                   1.664585e+11             1.798772e+11   
     9                   1.372683e+11             1.497610e+11   
     10                  1.263122e+11             1.394534e+11   
     11                  1.222004e+11             1.355091e+11   
     12                  1.427166e+11             1.570085e+11   

            elec fuel total co2 (kg)  
year month                            
2017 8                  1.691147e+11  
     9                  1.396417e+11  
     10                 1.288218e+11  
     11                 1.247023e+11  
     12                 1.453625e+11

### Total national generation (all fuels)
Add generation/fuel consumption reported by facilities (with state-level fuel codes) and extra generation/fuel consumption (which also uses state-level fuel codes).

In [34]:
national_gen = (gen_fuels_state
                .groupby(['type', 'year', 'month'])['generation (mwh)'].sum()
                .add(extra_gen_fuel['generation (mwh)'], fill_value=0))

In [35]:
national_gen.groupby(['year', 'month']).sum().tail()

year  month
2017  8        0.0
      9        0.0
      10       0.0
      11       0.0
      12       0.0
Name: generation (mwh), dtype: float64

#### Regroup generation from state codes to my custom fuel codes

In [36]:
national_gen = group_fuel_cats(national_gen.reset_index(), custom_fuel_cat,
                               'type', 'fuel category').set_index(['fuel category', 'year', 'month'])

In [37]:
national_gen.tail()

generation (mwh)
fuel category year month                  
Wind          2017 8                   0.0
                   9                   0.0
                   10                  0.0
                   11                  0.0
                   12                  0.0

In [38]:
total_gen = national_gen.groupby(['year', 'month']).sum()
total_gen.head()

generation (mwh)
year month                  
2001 1           332493160.0
     2           282940198.0
     3           300706544.0
     4           278078871.0
     5           300491621.0

### Percent generation by custom fuel type

In [39]:
df_list = []
for fuel in national_gen.index.get_level_values('fuel category').unique():
    percent_gen = national_gen.loc[fuel].divide(total_gen, fill_value=0)
    percent_gen['fuel category'] = fuel
    percent_gen.set_index('fuel category', inplace=True, append=True)
    df_list.append(percent_gen)
percent_gen = pd.concat(df_list)

In [40]:
path = join(data_path, 'National data', 'National generation 2018-03-06.csv')
national_gen.to_csv(path)

path = join(data_path, 'National data', 'National percent gen 2018-03-06.csv')
percent_gen.to_csv(path)

## Facility state and lat/lon file generation
~~Create a .csv with lat/lon and state code for each facility. Need to manually add the NERC region label.~~ ~~I've done this with a spatial join between the lat/lon and NERC shapefiles in QGIS. It should be possible to do the spatial join in [GeoPandas](http://geopandas.org/).  Could also add custom region labels (e.g. eGRID subregions, ISO/RTO boundaries, etc).~~

This isn't possible with a spatial join because NERC regions aren't strictly based on geography. Instead, I've taken the NERC codes from EIA-860 and assigned unknown plants (mostly those that retired before 2012 when modern NERCs were mostly defined, and those that were assigned a plant id in 2017) NERC labels using a k-nearest neighbors algorithm and lat/lon information.

See the `Assign NERC region labels` notebook for the creation of this file.

## Fraction of estimated gen/fuels in each NERC region

**NOTE** EIA changed the reporting requirements for wind and solar facilities in 2017. I'm now going to use the list of annual facilities from 2017 rather than from 2015, but still use the reported generation by those facilities in 2015.

In [41]:
%aimport Data.make_data
from Data.make_data import get_annual_plants

In [47]:
annual_ids_2015 = get_annual_plants(2015)

C:\Users\gschivley\Documents\GitHub\Index-variability\Data storage\EIA downloads\f923_2015.zip


C:\Users\gschivley\Anaconda2\envs\psci\lib\site-packages\pandas\util\_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


In [48]:
annual_ids_2017 = get_annual_plants(2017)

Combine the lists of annual plants in 2015 and 2017. This lets us catch facilities that have gone from monthly to annual since 2015, but it also includes plants that were annual in 2015 an may have retired.

There is the possibility of an error in allocation to NERC regions for 2016 state-level generation when using the facilities that changed to annual in 2017. But since the state-level generation is so much smaller in 2016 I don't think this is much of an issue.

In [49]:
annual_ids = set(annual_ids_2015.tolist() + annual_ids_2017.tolist())

In [50]:
len(annual_ids)

7491

### Add NERC region labels

In [51]:
%aimport Analysis.state2nerc
from Analysis.state2nerc import fraction_state2nerc, add_region

In [52]:
cwd = os.getcwd()
path = join(data_path, 'Facility labels',
            'Facility locations_knn.csv')
location_labels = pd.read_csv(path)

In [53]:
nerc_state_path = join(data_path, 'Derived data',
                  'NERC_states.json')

with open(nerc_state_path, 'r') as f:
    nerc_states = json.load(f)

Added the filter that year must be 2015 - was getting all 2015 annual plants, but for all years!

In [54]:
eia_2015_annual = eia_fac.loc[(eia_fac['plant id'].isin(annual_ids)) & 
                              (eia_fac['year'] == 2015)].copy()

# Group to state-level fuel categories
eia_2015_annual = group_fuel_cats(eia_2015_annual, state_fuel_cat)

In [55]:
eia_2015_annual_nerc = add_facility_location(eia_2015_annual, location_labels, 
                                        labels=['state', 'nerc'])

This is 2015 data on annual reporting facilities (from both 2015 and 2017)

In [56]:
eia_2015_annual_nerc.tail()

,type,year,month,plant id,total fuel (mmbtu),generation (mwh),elec fuel (mmbtu),lat,lon,quarter,all fuel fossil co2 (kg),elec fuel fossil co2 (kg),all fuel total co2 (kg),elec fuel total co2 (kg),state,nerc
92446,WWW,2015,8,58707,211987.0,13966.0,211987.0,33.238611,-80.450278,3,0.0,0.0,19884380.6,19884380.6,SC,RFC
92447,WWW,2015,9,58707,214505.0,13769.0,214505.0,33.238611,-80.450278,3,0.0,0.0,20120569.0,20120569.0,SC,RFC
92448,WWW,2015,10,58707,162426.0,8554.0,162426.0,33.238611,-80.450278,4,0.0,0.0,15235558.8,15235558.8,SC,RFC
92449,WWW,2015,11,58707,245845.0,13352.0,245845.0,33.238611,-80.450278,4,0.0,0.0,23060261.0,23060261.0,SC,RFC
92450,WWW,2015,12,58707,249553.0,14099.0,249553.0,33.238611,-80.450278,4,0.0,0.0,23408071.4,23408071.4,SC,RFC


In [57]:
# Get a list of all state abbreviations

all_states = []
for value in nerc_states.values():
    all_states.extend(value)
all_states = set(all_states)

In [58]:
df_list = []

for state in all_states:
    try:
        df_list.append(fraction_state2nerc(eia_2015_annual_nerc,
                       state, region_col='nerc', fuel_col='type'))
    except:
        print(state)
        pass

In [59]:
nerc_fraction = pd.concat(df_list)
nerc_fraction.set_index(['state', 'nerc', 'type'], inplace=True)
nerc_fraction.sort_index(inplace=True)

In [60]:
nerc_fraction.loc[idx['TX', :, 'WND'], :]

% generation  % total fuel  % elec fuel
state nerc type                                         
TX    SPP  WND       0.125231      0.125231     0.125231
      TRE  WND       0.874769      0.874769     0.874769
      WECC WND       0.000000      0.000000     0.000000

With the values below I can allocate extra state-level generation and fuel use to each of the NERC regions!

In [61]:
nerc_fraction.loc['TX']

% generation  % total fuel  % elec fuel
nerc type                                         
SERC HYC       0.260935      0.260935     0.260935
     NG        0.124994      0.153379     0.092145
     OOG       0.151901      0.546518     0.082765
     OTH      -0.006302      0.025416     0.006614
     WWW      -0.492566      0.190640     0.257302
SPP  NG        0.160722      0.174954     0.185692
     OOG       0.282734      0.159436     0.284516
     PEL       0.478368      0.745665     0.290000
     WAS       0.892327      0.958075     0.847920
     WND       0.125231      0.125231     0.125231
     WWW       0.609573      0.282686     0.398931
TRE  HYC       0.739065      0.739065     0.739065
     NG        0.652469      0.615575     0.650881
     OOG       0.565365      0.294046     0.632719
     OTH       1.006302      0.974584     0.993386
     PC        1.000000      1.000000     1.000000
     PEL       0.487887      0.245383     0.677393
     SUN       0.932089      0.932089     0.932089
     WAS       0.107673      0.041925     0.152080
     WND       0.874769      0.874769     0.874769
     WWW       0.882992      0.526674     0.343767
WECC NG        0.061815      0.056091     0.071281
     PEL       0.033746      0.008952     0.032607
     SUN       0.067911      0.067911     0.067911
     WND       0.000000      0.000000     0.000000

In [62]:
nerc_fraction.tail()

% generation  % total fuel  % elec fuel
state nerc type                                         
WY    WECC NG             1.0           1.0          1.0
           OOG            1.0           1.0          1.0
           OTH            1.0           1.0          1.0
           PEL            1.0           1.0          1.0
           WND            1.0           1.0          1.0

Making sure that no values are greater than 1 (within tolerance)

In [63]:
(nerc_fraction.groupby(['state', 'type']).sum() > 1.0001).any()

% generation    False
% total fuel    False
% elec fuel     False
dtype: bool

In [64]:
(nerc_fraction.groupby(['state', 'type']).sum()
 .loc[(nerc_fraction.groupby(['state', 'type']).sum() > 1).any(axis=1)])

,,% generation,% total fuel,% elec fuel
state,type,,,
AR,NG,1.0,1.0,1.0
TX,PEL,1.0,1.0,1.0


## Allocate extra gen from the state-level to regions

I still need to generate state-level total generation and fuel use!

In [65]:
idx = pd.IndexSlice

In [66]:
# a dictionary to match column names
nerc_frac_match = {'% generation': 'generation (mwh)',
                   '% total fuel': 'total fuel (mmbtu)',
                   '% elec fuel': 'elec fuel (mmbtu)'}

### Load state-level total gen/fuel consumption

In [67]:
path = join(data_path, 'EIA state-level gen fuel CO2.csv')

state_total = pd.read_csv(path, parse_dates=['datetime'])

In [68]:
rename_cols(state_total)
state_total['state'] = state_total['geography'].str[-2:]

In [69]:
state_total.head()

,type,year,month,geography,end,f,last_updated,sector,series_id,start,units,generation (mwh),total fuel (mmbtu),elec fuel (mmbtu),all fuel co2 (kg),elec fuel co2 (kg),datetime,quarter,state
0,AOR,2001,1,USA-AK,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-AK-99.M,200101.0,megawatthours,87.00,NaN,NaN,NaN,NaN,2001-01-01,1,AK
1,AOR,2001,1,USA-AL,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-AL-99.M,200101.0,megawatthours,401167.59,NaN,NaN,NaN,NaN,2001-01-01,1,AL
2,AOR,2001,1,USA-AR,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-AR-99.M,200101.0,megawatthours,136530.37,NaN,NaN,NaN,NaN,2001-01-01,1,AR
3,AOR,2001,1,USA-AZ,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-AZ-99.M,200101.0,megawatthours,453.00,NaN,NaN,NaN,NaN,2001-01-01,1,AZ
4,AOR,2001,1,USA-CA,201706.0,M,2017-08-24T11:46:12-04:00,99.0,ELEC.GEN.AOR-CA-99.M,200101.0,megawatthours,1717398.41,NaN,NaN,NaN,NaN,2001-01-01,1,CA


In [70]:
state_total.dtypes

type                          object
year                           int64
month                          int64
geography                     object
end                          float64
f                             object
last_updated                  object
sector                       float64
series_id                     object
start                        float64
units                         object
generation (mwh)             float64
total fuel (mmbtu)           float64
elec fuel (mmbtu)            float64
all fuel co2 (kg)            float64
elec fuel co2 (kg)           float64
datetime              datetime64[ns]
quarter                        int64
state                         object
dtype: object

Simplify the dataframe

In [71]:
cols = list(nerc_frac_match.values())
state_total = state_total.groupby(['state', 'year', 'month', 'type'])[cols].sum()
state_total.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state year month type                                                         
AK    2001 1     AOR               87.0                 0.0                0.0
                 COW            46903.0           1120000.0           872000.0
                 HYC           104549.0                 0.0                0.0
                 NG            367521.0           4091000.0          3989000.0
                 PEL            71085.0            767000.0           763000.0

In [72]:
# list of NERC regions
nercs = nerc_fraction.index.get_level_values('nerc').unique()

### Group the facility data to state fuel categories and add state labels

In [73]:
cols = list(nerc_frac_match.values())
eia_fac_type = group_fuel_cats(eia_fac, state_fuel_cat)
eia_fac_type = add_facility_location(eia_fac_type, location_labels, ['state'])
eia_fac_type = eia_fac_type.groupby(['state', 'year', 'month', 'type'])[cols].sum()

In [74]:
eia_fac_type.loc[idx['OK', 2017, :, 'WND'], :]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state year month type                                                         
OK    2017 1     WND           982955.0           9074640.0          9074640.0
           2     WND          1053595.0           9726789.0          9726789.0
           3     WND          1237665.0          11426122.0         11426122.0
           4     WND          1164423.0          10749954.0         10749954.0
           5     WND           960103.0           8863670.0          8863670.0
           6     WND           987307.0           9114817.0          9114817.0
           7     WND           741231.0           6843046.0          6843046.0
           8     WND           494247.0           4562888.0          4562888.0
           9     WND           922794.0           8519233.0          8519233.0
           10    WND          1280034.0          11817274.0         11817274.0
           11    WND          1012855.0           9350678.0          9350678.0
           12    WND          1024590.0           9459017.0          9459017.0

In [75]:
eia_fac_type.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
state year month type                                                         
AK    2001 1     COW          26493.302           576574.00          520312.00
                 HYC         111786.609          1155091.03         1155091.03
                 NG          314206.726          3372210.00         3270620.00
                 PEL          66098.653           715175.00          710849.00
                 WND             86.518              893.99             893.99

### Calculate the extra gen/fuel consumption at the state levels
**Only worrying about extra generation from 2016 forward.**

In [76]:
state_extra = (state_total.loc[idx[:, 2016:, :, :], :]
               - eia_fac_type.loc[idx[:, 2016:, :, :], :])
state_extra.dropna(how='all', inplace=True)
state_extra = state_extra.reorder_levels(['year', 'state', 'month', 'type'])
state_extra.sort_index(inplace=True)
state_extra.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year state month type                                                         
2016 AK    1     COW           -237.340            -54290.0            46511.0
                 HYC         -18155.827          -1431529.0         -1431529.0
                 NG          -89791.999           -797117.0          -748132.0
                 PEL           5192.862             57395.0            60259.0
                 WAS           4927.814              -364.0             -364.0

The huge jump in Texas state-level wind generation is why I need to use more recent lists of annual reporting facilities rather than the list from 2015.

In [77]:
state_extra.loc[idx[:, 'TX', :, 'WND'], :]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year state month type                                                         
2016 TX    1     WND           2185.074         -41090874.0        -41090874.0
           2     WND           1487.947         -47271944.0        -47271944.0
           3     WND          12598.793         -52024897.0        -52024897.0
           4     WND          13508.958         -43730543.0        -43730543.0
           5     WND         -75977.729         -47756097.0        -47756097.0
           6     WND          37651.286         -34911405.0        -34911405.0
           7     WND           -776.804         -52390783.0        -52390783.0
           8     WND           9331.168         -34177371.0        -34177371.0
           9     WND           8061.550         -36147145.0        -36147145.0
           10    WND          21765.422         -50325557.0        -50325557.0
           11    WND            152.159         -41694489.0        -41694489.0
           12    WND         -10259.434         -49603096.0        -49603096.0
2017 TX    1     WND        2989930.730         -26838016.0        -26838016.0
           2     WND        2921951.520         -26732669.0        -26732669.0
           3     WND        3449182.090         -32696864.0        -32696864.0
           4     WND        3443022.330         -31905751.0        -31905751.0
           5     WND        3043864.570         -29618190.0        -29618190.0
           6     WND        2408185.790         -22552789.0        -22552789.0

Why is generation from facilities greater than state-level estimates in some cases?

In [78]:
state_extra.loc[(state_extra < -100).any(axis=1)]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year state month type                                                         
2016 AK    1     COW           -237.340            -54290.0            46511.0
                 HYC         -18155.827          -1431529.0         -1431529.0
                 NG          -89791.999           -797117.0          -748132.0
                 WAS           4927.814              -364.0             -364.0
                 WND          10638.938            -41252.0           -41252.0
           2     COW           -691.456            -54018.0            44456.0
                 HYC          -4604.416          -1180430.0         -1180430.0
                 NG          -73438.381           -651795.0          -609250.0
                 PEL         -10933.455            -93881.0           -87142.0
                 WAS           4505.428              -264.0             -264.0
                 WND           7246.528            -77713.0           -77713.0
           3     COW           -358.386            -37729.0            47048.0
                 HYC          -8709.399          -1508803.0         -1508803.0
                 NG          -89371.973           -781162.0          -736368.0
                 PEL         -10966.946           -100824.0           -95404.0
                 WAS           5342.949              -448.0             -448.0
                 WND           5933.420            -90689.0           -90689.0
           4     COW          -8463.447           -182197.0           -86323.0
                 HYC          24947.326           -946916.0          -946916.0
                 NG         -102022.618           -940708.0          -886100.0
                 PEL          -3268.069            -22276.0           -24736.0
                 WAS           5021.483              -424.0             -424.0
                 WND          -3106.445           -175635.0          -175635.0
           5     COW           -711.692            -12842.0            25592.0
                 HYC          40126.830           -627194.0          -627194.0
                 NG          -89565.104           -805944.0          -758576.0
                 WAS           4208.350              -460.0             -460.0
                 WND          -1262.681           -128888.0          -128888.0
           6     COW          -7144.893           -102553.0           -37766.0
                 HYC          26968.877           -727130.0          -727130.0
...                                 ...                 ...                ...
2017 WI    6     OTH           3043.884            -31923.0           -25606.0
                 WAS          44046.389            -28165.0           -26127.0
                 WND         133771.730             -5216.0            -5216.0
                 WWW          47880.040           -785708.0          -199805.0
     WV    1     OOG              0.005            -43295.0           -10033.0
                 WND          68617.720           -431245.0          -431245.0
           2     OOG              0.004            -43123.0           -10226.0
                 WND          87721.730           -278197.0          -278197.0
           3     OOG             -0.001            -52401.0           -11878.0
                 WND         109273.480           -546128.0          -546128.0
           4     OOG              0.003            -36187.0            -8006.0
                 WND         118877.250           -498814.0          -498814.0
           5     OOG             -0.004            -49183.0           -11719.0
                 WND          91086.370           -518404.0          -518404.0
           6     OOG             -0.001            -54522.0           -13484.0
                 WND         100375.100           -373979.0          -373979.0
     WY    1     OOG            193.229           -688106.0          -235324.0
                 OTH             -0.004           -242464.0           -24

Sort the index of each dataframe to make sure they can be easily combined.

In [79]:
nerc_fraction.sort_index(inplace=True)
nerc_fraction.loc[idx['TX', 'WECC', :], :].tail()

% generation  % total fuel  % elec fuel
state nerc type                                         
TX    WECC NG        0.061815      0.056091     0.071281
           PEL       0.033746      0.008952     0.032607
           SUN       0.067911      0.067911     0.067911
           WND       0.000000      0.000000     0.000000

In [80]:
state_extra.sort_index(inplace=True)
state_extra.head()

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year state month type                                                         
2016 AK    1     COW           -237.340            -54290.0            46511.0
                 HYC         -18155.827          -1431529.0         -1431529.0
                 NG          -89791.999           -797117.0          -748132.0
                 PEL           5192.862             57395.0            60259.0
                 WAS           4927.814              -364.0             -364.0

Create a copy of the `nerc_fraction` dataframe with repeated values for every month of the year, so that they MultiIndex matches the `state_extra` MultiIndex

In [81]:
df_list = []
for month in range(1, 13):
    df = nerc_fraction.copy()
    df['month'] = month
    df.set_index('month', append=True, inplace=True)
    df_list.append(df)

nerc_frac_monthly = pd.concat(df_list, axis=0)
nerc_frac_monthly.sort_index(inplace=True)
nerc_frac_monthly = (nerc_frac_monthly
                     .reorder_levels(['nerc', 'state', 'month', 'type']))

In [82]:
nercs

Index(['SERC', 'SPP', 'WECC', 'MRO', 'NPCC', 'RFC', 'FRCC', 'TRE'], dtype='object', name='nerc')

Cycle through each year (2016 and 2017 in this case) and each NERC, multiplying the state-level extra generation, total fuel consumption, and fuel consumption for electricity by the share that should be allocated to each NERC.

In [83]:
df_list_outer = []
for year in [2016, 2017]:
    df_list_inner = []
    for nerc in nercs:
        df = pd.concat([(nerc_frac_monthly
                         .loc[nerc]['% generation']
                         * state_extra
                         .loc[year]['generation (mwh)']).dropna(),
                        (nerc_frac_monthly.
                         loc[nerc]['% total fuel']
                         * state_extra
                         .loc[year]['total fuel (mmbtu)']).dropna(),
                        (nerc_frac_monthly
                         .loc[nerc]['% elec fuel']
                         * state_extra
                         .loc[year]['elec fuel (mmbtu)']).dropna()],
                        axis=1)
        df.columns = nerc_frac_match.values()
        df['nerc'] = nerc
        df['year'] = year
        df = df.groupby(['year', 'nerc', 'month', 'type']).sum()
        df_list_inner.append(df)

    df_list_outer.append(pd.concat(df_list_inner))
final = pd.concat(df_list_outer)
final.sort_index(inplace=True)

In [84]:
nerc_frac_monthly.sort_index(inplace=True)

Although the dataframe is called `final`, it's really just the final allocated extra state-level generation/fuel consumption

In [85]:
final.loc[idx[2017, 'SPP', :, 'WND'], :]

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year nerc month type                                                         
2017 SPP  1     WND       2.210603e+06       -1.957314e+07      -1.957314e+07
          2     WND       2.227095e+06       -2.033684e+07      -2.033684e+07
          3     WND       2.612247e+06       -2.547078e+07      -2.547078e+07
          4     WND       2.556985e+06       -2.517868e+07      -2.517868e+07
          5     WND       2.221616e+06       -2.157526e+07      -2.157526e+07
          6     WND       1.835518e+06       -2.024732e+07      -2.024732e+07

In [86]:
final

generation (mwh)  total fuel (mmbtu)  elec fuel (mmbtu)
year nerc month type                                                         
2016 FRCC 1     COW       1.077128e+03        2.382266e+05       4.286583e+03
                NG       -1.720383e+04       -1.275135e+05       4.393151e+04
                OOG      -2.105648e+04       -3.029594e+05      -2.814671e+05
                OTH       4.524713e+02       -3.980957e+06      -3.690502e+06
                PEL       1.479510e+03        1.638231e+04       1.439410e+04
                SUN       1.543550e+02       -8.118300e+04      -8.118300e+04
                WAS       5.815508e+04       -3.356567e+06      -3.090012e+06
                WWW      -3.173020e+04       -9.490979e+06      -1.582372e+06
          2     COW       5.095846e+02        2.265000e+05       1.921827e+03
                NG       -1.722224e+04       -8.125604e+04       2.109770e+04
                OOG      -2.347692e+04       -3.350667e+05      -3.102727e+05
                OTH       8.393979e+03       -3.593557e+06      -3.334421e+06
                PEL      -6.874316e+02       -6.834360e+04      -6.396933e+02
                SUN       2.169625e+03       -1.402670e+05      -1.402670e+05
                WAS       5.544121e+04       -3.204712e+06      -2.942827e+06
                WWW      -4.046758e+04       -9.199629e+06      -1.492875e+06
          3     COW      -1.618798e+03        9.342500e+04      -4.396440e+04
                HYC       2.742592e+03       -1.849460e+05      -1.849460e+05
                NG       -7.025320e+03       -7.689769e+04       9.232017e+04
                OOG      -2.531292e+04       -3.477942e+05      -3.260615e+05
                OTH      -3.214277e+03       -3.827666e+06      -3.556679e+06
                PEL      -1.329807e+03       -1.083976e+05      -6.741768e+03
                SUN       3.408456e+03       -1.538000e+05      -1.538000e+05
                WAS       5.388957e+04       -3.179556e+06      -2.854240e+06
                WWW      -3.994904e+04       -9.550917e+06      -1.510796e+06
          4     COW      -5.644118e+03       -3.332591e+04      -1.393389e+05
                NG        1.361463e+04        2.212693e+04       2.052274e+05
                OOG      -2.566410e+04       -3.617726e+05      -3.257619e+05
                OTH      -1.173528e+03       -4.120628e+06      -3.901671e+06
                PEL      -9.102219e+02       -8.378612e+04      -4.845677e+03
...                                ...                 ...                ...
2017 WECC 4     NG        1.187362e+06        1.218251e+07       1.135868e+07
                OOG      -8.100281e+04       -4.092450e+06      -1.950956e+06
                OTH       2.297305e+03       -1.502707e+06      -1.251310e+06
                PEL       1.265991e+03        6.154001e+04       1.525211e+04
                SUN       2.203562e+06       -1.039280e+07      -1.039280e+07
                WAS       2.323007e+05       -9.206360e+05      -6.300050e+05
                WND       2.901120e+06       -1.256139e+07      -1.256139e+07
                WWW       3.478885e+05       -3.164764e+06      -1.640745e+06
          5     COW       1.850173e+05        2.130796e+06       1.646581e+06
                GEO       7.202312e+05       -5.832130e+06      -5.832130e+06
                HYC       5.549492e+06       -1.345381e+08      -1.345381e+08
                NG        1.138999e+06        1.202059e+07       1.136572e+07
                OOG      -8.268553e+04       -4.213459e+06      -1.983039e+06
                OTH       2.114460e+03       -1.293918e+06      -1.090821e+06
                PEL       6.493692e+02        9.668846e+04       2.288388e+04
                SUN       2.547555e+06       -1.258208e+07      -1.258208e+07
                WAS       2.412384e+05       -9.792870e+05      -6.884040e+05
                WND       2.552278e+06       -1.120687e+07      -1.120687e+07
                WWW       3.672885e+05      

In [87]:
path = join(data_path, 'NERC extra gen fuels 2018-03-06.csv')
final.to_csv(path)